Imports. The output of `tf.__version__` should read something like 2.#

In [2]:
import os
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector
import pycocotools
import tensorflow as tf
from absl import flags
from pathlib import Path
print(tf.__version__)
import logging
logging.basicConfig(level=logging.INFO)
from tensorflow.python.client import device_lib
for d in device_lib.list_local_devices():
    print(d)
    print('---')

2.8.0
name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18373288242267729391
xla_global_id: -1

---
name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7765491712
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3049412725683231859
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1070, pci bus id: 0000:04:00.0, compute capability: 6.1"
xla_global_id: 416903419

---


2022-04-20 01:50:44.761814: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1900] Ignoring visible gpu device (device: 1, name: NVS 510, pci bus id: 0000:03:00.0, compute capability: 3.0) with Cuda compute capability 3.0. The minimum required Cuda capability is 3.5.
2022-04-20 01:50:44.762993: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /device:GPU:0 with 7405 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1070, pci bus id: 0000:04:00.0, compute capability: 6.1


Set directories.

In [3]:
DATA_DIR = Path("./data/")
TRAIN_FILE = DATA_DIR / "10k_train.tfrecord"
TEST_FILE = DATA_DIR / "10k_test.tfrecord"
CLASS_LABELS_FILE = DATA_DIR / "classlabels.txt"
CKPT_DIR = Path("./checkpoints") / "ckpts01"
EXPORT_DIR = Path("./export/") / "01"
os.makedirs(CKPT_DIR, exist_ok=True)
os.makedirs(EXPORT_DIR, exist_ok=True)

print("Reading data from ", DATA_DIR.resolve())
print("Saving Checkpoints to ", CKPT_DIR.resolve())
print("Exporting to ", EXPORT_DIR.resolve())

Reading data from  /home/ms/prg/train_targetID/data
Saving Checkpoints to  /home/ms/prg/train_targetID/checkpoints/ckpts01
Exporting to  /home/ms/prg/train_targetID/export/01


Read class labels.

In [4]:
with open(CLASS_LABELS_FILE, "r") as f:
    label_map = [s.strip() for s in f.readlines()]
    
label_map_dict = {}
for i, l in enumerate(label_map):
    label_map_dict[i+1] = l

for k, v in label_map_dict.items():
    print(k, ": ", v)

1 :  circle
2 :  diamond
3 :  halfcircle
4 :  heart
5 :  hexagon
6 :  hexstar
7 :  moon
8 :  octagon
9 :  pentagon
10 :  pentstar
11 :  plus
12 :  square
13 :  trapezoid
14 :  triangle


In [5]:
train_data = object_detector.DataLoader(
    tfrecord_file_patten = str(TRAIN_FILE.resolve()),
    size = 9000,
    label_map = label_map_dict
)

test_data = object_detector.DataLoader(
    tfrecord_file_patten = str(TEST_FILE.resolve()),
    size = 1000,
    label_map = label_map_dict
)

Create spec with hyperparameters.

In [6]:
hparams = {
    "map_freq" : 2,
    "learning_rate" : 0.2,
}
spec = object_detector.EfficientDetLite1Spec(
    hparams=hparams, model_dir=CKPT_DIR, epochs=100, batch_size=64,
    var_freeze_expr='(efficientnet)',
    tflite_max_detections = 10,
    verbose = 1,
)

2022-04-20 01:50:56.315577: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1900] Ignoring visible gpu device (device: 1, name: NVS 510, pci bus id: 0000:03:00.0, compute capability: 3.0) with Cuda compute capability 3.0. The minimum required Cuda capability is 3.5.
2022-04-20 01:50:56.323517: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7405 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1070, pci bus id: 0000:04:00.0, compute capability: 6.1


Train model

In [ ]:
model = object_detector.create(
    train_data = train_data,
    model_spec = spec,
    validation_data = test_data,
    train_whole_model = False,
    do_train = True,
)
model.export(
    export_dir = EXPORT_DIR,
    export_format = [ExportFormat.TFLITE, ExportFormat.LABEL, ExportFormat.SAVED_MODEL]
)

INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...
INFO:absl:Using /tmp/tfhub_modules to cache modules.
INFO:absl:target_size = (384, 384), output_size = (384, 384)
INFO:absl:LR schedule method: cosine
INFO:absl:Use SGD optimizer


Epoch 1/100


2022-04-20 01:51:39.633049: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8400
2022-04-20 01:51:40.694170: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


140/140 [==============================] - ETA: 0s - det_loss: 1.1020 - cls_loss: 0.8498 - box_loss: 0.0050 - reg_l2_loss: 0.0188 - loss: 1.1208 - learning_rate: 0.1033 - gradient_norm: 0.9456
Epoch 1: saving model to checkpoints/ckpts01/ckpt-1
140/140 [==============================] - 167s 984ms/step - det_loss: 1.0988 - cls_loss: 0.8474 - box_loss: 0.0050 - reg_l2_loss: 0.0188 - loss: 1.1176 - learning_rate: 0.1040 - gradient_norm: 0.9464 - val_det_loss: 1.1973 - val_cls_loss: 1.0138 - val_box_loss: 0.0037 - val_reg_l2_loss: 0.0195 - val_loss: 1.2168
Epoch 2/100
140/140 [==============================] - ETA: 0s - det_loss: 0.5572 - cls_loss: 0.4423 - box_loss: 0.0023 - reg_l2_loss: 0.0207 - loss: 0.5779 - learning_rate: 0.1999 - gradient_norm: 0.8597
Epoch 2: saving model to checkpoints/ckpts01/ckpt-2
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, m

...

DONE (t=4.25s).
Accumulating evaluation results...
DONE (t=0.09s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.608
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.662
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.657
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.053
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.761
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.836
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.007
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.066
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.621
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.055
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.780
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= lar

Epoch 65/100
140/140 [==============================] - ETA: 0s - det_loss: 0.1934 - cls_loss: 0.1461 - box_loss: 9.4458e-04 - reg_l2_loss: 0.0429 - loss: 0.2362 - learning_rate: 0.0542 - gradient_norm: 0.4191
Epoch 65: saving model to checkpoints/ckpts01/ckpt-65
140/140 [==============================] - 140s 997ms/step - det_loss: 0.1931 - cls_loss: 0.1459 - box_loss: 9.4294e-04 - reg_l2_loss: 0.0429 - loss: 0.2360 - learning_rate: 0.0542 - gradient_norm: 0.4183 - val_det_loss: 0.1660 - val_cls_loss: 0.1317 - val_box_loss: 6.8558e-04 - val_reg_l2_loss: 0.0428 - val_loss: 0.2088
Epoch 66/100
140/140 [==============================] - ETA: 0s - det_loss: 0.1912 - cls_loss: 0.1442 - box_loss: 9.3878e-04 - reg_l2_loss: 0.0428 - loss: 0.2339 - learning_rate: 0.0514 - gradient_norm: 0.4160
Epoch 66: saving model to checkpoints/ckpts01/ckpt-66
creating index...
index created!
Loading and preparing results...
Converting ndarray to lists...
(96000, 7)
0/96000
DONE (t=0.77s)
creating index...


DONE (t=0.81s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=4.23s).
Accumulating evaluation results...
DONE (t=0.09s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.611
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.658
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.656
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.051
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.757
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.860
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.007
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.067
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.622
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.051
 Average Recall     (

Epoch 79/100
140/140 [==============================] - ETA: 0s - det_loss: 0.1819 - cls_loss: 0.1362 - box_loss: 9.1389e-04 - reg_l2_loss: 0.0415 - loss: 0.2234 - learning_rate: 0.0204 - gradient_norm: 0.4182
Epoch 79: saving model to checkpoints/ckpts01/ckpt-79
140/140 [==============================] - 140s 999ms/step - det_loss: 0.1820 - cls_loss: 0.1363 - box_loss: 9.1512e-04 - reg_l2_loss: 0.0415 - loss: 0.2235 - learning_rate: 0.0204 - gradient_norm: 0.4184 - val_det_loss: 0.1509 - val_cls_loss: 0.1191 - val_box_loss: 6.3671e-04 - val_reg_l2_loss: 0.0415 - val_loss: 0.1924
Epoch 80/100
140/140 [==============================] - ETA: 0s - det_loss: 0.1797 - cls_loss: 0.1350 - box_loss: 8.9452e-04 - reg_l2_loss: 0.0414 - loss: 0.2212 - learning_rate: 0.0186 - gradient_norm: 0.4110
Epoch 80: saving model to checkpoints/ckpts01/ckpt-80
creating index...
index created!
Loading and preparing results...
Converting ndarray to lists...
(96000, 7)
0/96000
DONE (t=0.77s)
creating index...


DONE (t=0.77s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=4.27s).
Accumulating evaluation results...
DONE (t=0.09s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.622
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.665
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.659
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.051
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.776
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.865
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.007
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.067
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.631
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.052
 Average Recall     (